In [10]:
!pip install "pennylane-lightning[gpu]" --upgrade
!pip install cuquantum-python --upgrade


/opt/conda/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Using cached pennylane_lightning-0.43.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached pennylane-0.43.0-py3-none-any.whl.metadata (11 kB)
  Using cached scipy_openblas32-0.3.30.0.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (57 kB)
  Using cached scipy-1.16.2-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached autograd-1.8.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached autoray-0.8.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached diastatic_malt-2.15.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse

In [1]:
import pennylane as qml
dev = qml.device("lightning.gpu", wires=1)
print("OK", dev)


/home/jovyan/.qbraid/environments/h_mmigxh/pyenv/lib/python3.11/site-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.8.0 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


OK <lightning.gpu device (wires=1) at 0x7cc82c1276d0>


In [8]:
!pip install jax

  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp311-cp311-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached ml_dtypes-0.5.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
Using cached jax-0.8.0-py3-none-any.whl (2.9 MB)
Using cached jaxlib-0.8.0-cp311-cp311-manylinux_2_27_x86_64.whl (79.7 MB)
Using cached ml_dtypes-0.5.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (4.9 MB)
Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [jax]3/4 [jax]ib]


In [ ]:
# Jax
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
import jax
import jax.numpy as jnp
import optax  # JAX 生态中的主流优化器库
import itertools, time
from collections import defaultdict, Counter
import os
import sys

# ----------------------------
# JAX 
# ----------------------------
# JAX 
#  "lightning.gpu" or PennyLane NVIDIA  cuQuantum SDK

WIRE_COUNT = 9
try:
    dev = qml.device("lightning.gpu", wires=WIRE_COUNT)
    print("成功加载 lightning.gpu 设备，将使用 NVIDIA cuQuantum 加速。")
except (qml.DeviceError, ImportError):
    print("lightning.gpu 不可用，回退到 lightning.qubit。")
    dev = qml.device("lightning.qubit", wires=WIRE_COUNT)

# ----------------------------
# model
# ----------------------------
#  Python list or np.array not jnp.array
corners = [0, 2, 6, 8]
edges = [1, 3, 5, 7]
center = 4

# increase the model size

n_layers = 2
p_repetitions = 2
params_per_block = 6
total_params = (n_layers + 1) * p_repetitions * params_per_block

rng = np.random.default_rng(42)

def is_valid_game(board):
    x_count = np.sum(board == 1)
    o_count = np.sum(board == -1)
    return (x_count == o_count) or (x_count == o_count + 1)

def check_winner_terminal(board):
    lines = [
        [0,1,2],[3,4,5],[6,7,8],
        [0,3,6],[1,4,7],[2,5,8],
        [0,4,8],[2,4,6]
    ]
    board_tuple = tuple(board)
    for line in lines:
        if board_tuple[line[0]] == board_tuple[line[1]] == board_tuple[line[2]] == 1:
            return 'cross'
        if board_tuple[line[0]] == board_tuple[line[1]] == board_tuple[line[2]] == -1:
            return 'circle'
    return 'draw'

def generate_all_valid_games(subset_size=None):
    games = []
    # increase size
    for num_pieces in range(10): 
        for positions in itertools.combinations(range(9), num_pieces):
            num_x = (num_pieces + 1) // 2
            for x_pos in itertools.combinations(positions, num_x):
                board = np.zeros(9, dtype=np.int8)
                o_pos = list(set(positions) - set(x_pos))
                board[list(x_pos)] = 1
                board[o_pos] = -1
                label = check_winner_terminal(board)
                games.append((board, label))
    
    unique_games_set = set([(tuple(b), l) for b, l in games])
    games = [(np.array(b), l) for b, l in unique_games_set]
    
    if subset_size is not None and subset_size < len(games):
        idx = rng.choice(len(games), size=subset_size, replace=False)
        games = [games[i] for i in idx]
        
    rng.shuffle(games)
    return games


def data_encoding_pennylane(board):
    for i, v in enumerate(board):
        qml.RZ(v * jnp.pi, wires=i)
        qml.RY(v * (jnp.pi/2), wires=i)

def branch_block_center(params):
    _, _, th_m, th_ce, th_ec, _ = params
    qml.RY(th_m, wires=center)
    for c in corners:
        qml.CRY(th_ce/6, wires=[center, c])
    for e in edges:
        qml.CRY(th_ec/6, wires=[center, e])

def branch_block_corners(params):
    th_c, _, _, _, _, th_cm = params
    for q in corners:
        qml.RY(th_c, wires=q)
    pairs = [(0,2),(2,8),(8,6),(6,0)]
    for a,b in pairs:
        qml.CZ(wires=[a,b])
    for c in corners:
        qml.CRY(th_cm/2, wires=[center, c])

def branch_block_edges(params):
    _, th_e, _, _, th_ec, _ = params
    for q in edges:
        qml.RY(th_e, wires=q)
    pairs = [(1,3),(3,7),(7,5),(5,1)]
    for a,b in pairs:
        qml.CZ(wires=[a,b])
    for e in edges:
        qml.CRY(th_ec/2, wires=[center, e])

def hierarchical_control_example(control_strength=0.1):
    for c in corners:
        qml.CRY(control_strength, wires=[center, c])
    for e in edges:
        qml.CRY(control_strength, wires=[center, e])


@qml.qnode(dev, interface='jax')
def qnode_tree(board, params):
    data_encoding_pennylane(board)
    param_idx = 0
    for _ in range(p_repetitions):
        block_params = jax.lax.dynamic_slice_in_dim(params, param_idx, 6, axis=0)
        branch_block_center(block_params)
        hierarchical_control_example(control_strength=0.05)
        branch_block_corners(block_params)
        branch_block_edges(block_params)
        param_idx += 6

    for _ in range(n_layers):
        data_encoding_pennylane(board)
        for _ in range(p_repetitions):
            block_params = jax.lax.dynamic_slice_in_dim(params, param_idx, 6, axis=0)
            branch_block_center(block_params)
            hierarchical_control_example(control_strength=block_params[5]*0.05)
            branch_block_corners(block_params)
            branch_block_edges(block_params)
            param_idx += 6
    
    return [qml.expval(qml.PauliZ(i)) for i in range(WIRE_COUNT)]


label_to_idx = {'circle':0, 'draw':1, 'cross':2}
idx_to_label = {v: k for k, v in label_to_idx.items()}

def to_logits_from_qnode_output(qnode_out):
    corner_expvals = qnode_out[jnp.array(corners)]
    edge_expvals = qnode_out[jnp.array(edges)]
    
    corner_avg = jnp.mean(corner_expvals)
    center_val = qnode_out[center]
    edge_avg = jnp.mean(edge_expvals)
    
    return jnp.stack([corner_avg, center_val, edge_avg])

batched_qnode = jax.vmap(qnode_tree, in_axes=(0, None))
batched_logits_computer = jax.vmap(to_logits_from_qnode_output, in_axes=(0))

def compute_logits(boards, params):
    qnode_outputs = jnp.array(batched_qnode(boards, params)).T
    return batched_logits_computer(qnode_outputs)

@jax.jit
def loss_fn(params, boards, labels, class_weights_arr):
    logits = compute_logits(boards, params)
    one_hot_labels = jax.nn.one_hot(labels, num_classes=3)
    weights = jnp.sum(one_hot_labels * class_weights_arr, axis=1)
    unweighted_loss = optax.softmax_cross_entropy(logits, one_hot_labels)
    weighted_loss = unweighted_loss * weights
    return jnp.mean(weighted_loss)

@jax.jit
def accuracy_fn(params, boards, labels):
    logits = compute_logits(boards, params)
    predictions = jnp.argmax(logits, axis=1)
    return jnp.mean(predictions == labels)


print("Generating...")
all_games = generate_all_valid_games()
print(f"Total：{len(all_games)}")

games_by_class = defaultdict(list)
for b, l in all_games:
    games_by_class[l].append(b)

counts = {lab: len(games_by_class[lab]) for lab in games_by_class}
total = sum(counts.values())
class_weights_map = {lab: total / (len(counts) * count) for lab, count in counts.items() if count > 0}
print("Counts:", counts)
print("Used Class Weights:", class_weights_map)

class_weights_arr = jnp.array([
    class_weights_map.get('circle', 1.0),
    class_weights_map.get('draw', 1.0),
    class_weights_map.get('cross', 1.0)
])

test_boards, test_labels, train_boards, train_labels = [], [], [], []
for lab, idx in label_to_idx.items():
    boards = games_by_class.get(lab, [])
    if not boards: continue
    rng.shuffle(boards)
    
    test_split_idx = len(boards) // 5
    test_boards.extend(boards[:test_split_idx])
    test_labels.extend([idx] * len(boards[:test_split_idx]))
    
    train_boards.extend(boards[test_split_idx:])
    train_labels.extend([idx] * len(boards[test_split_idx:]))

if train_labels:
    class_counts = Counter(train_labels)
    max_class_size = max(class_counts.values()) if class_counts else 0
    balanced_train_boards, balanced_train_labels = [], []
    for lab, idx in label_to_idx.items():
        class_boards = [b for b, l in zip(train_boards, train_labels) if l == idx]
        if not class_boards: continue
        
        repeats = (max_class_size + len(class_boards) - 1) // len(class_boards)
        resampled = (class_boards * repeats)[:max_class_size]
        
        balanced_train_boards.extend(resampled)
        balanced_train_labels.extend([idx] * max_class_size)

    train_boards = jnp.array(balanced_train_boards)
    train_labels = jnp.array(balanced_train_labels)
else:
    train_boards, train_labels = jnp.array([]), jnp.array([])

test_boards = jnp.array(test_boards)
test_labels = jnp.array(test_labels)

print(f"Train size: {len(train_boards)}, Validation size: {len(test_boards)}")
print(f"Train distribution: {Counter(np.array(train_labels))}")
print(f"Validation distribution : {Counter(np.array(test_labels))}")
# ----------------------------
# 训练循环
# ----------------------------
key = jax.random.PRNGKey(0)
params = jax.random.uniform(key, (total_params,), minval=-0.05, maxval=0.05)

optimizer = optax.adam(learning_rate=0.01)
opt_state = optimizer.init(params)
grad_fn = jax.value_and_grad(loss_fn)

batch_size = 128
epochs = 15

if len(train_boards) > 0 and len(test_boards) > 0:
    print("JIT compliing...")
    dummy_boards = train_boards[:2]
    dummy_labels = train_labels[:2]
    _ = loss_fn(params, dummy_boards, dummy_labels, class_weights_arr)
    _ = accuracy_fn(params, dummy_boards, dummy_labels)
    print("complied。Start training。")
    
    start_all = time.time()
    for epoch in range(1, epochs + 1):
        t_epoch = time.time()
        
        perms = jax.random.permutation(key, len(train_boards))
        key, _ = jax.random.split(key)
        
        batch_losses = []
        num_batches = len(train_boards) // batch_size
        
        
        for i in range(0, len(train_boards), batch_size):
           
            batch_num = i // batch_size + 1
            sys.stdout.write(f"\rEpoch {epoch}/{epochs} - Batch {batch_num}/{num_batches}")
            sys.stdout.flush()

            batch_idx = perms[i:i+batch_size]
            batch_boards = train_boards[batch_idx]
            batch_labels = train_labels[batch_idx]
            
            loss_val, grads = grad_fn(params, batch_boards, batch_labels, class_weights_arr)
            updates, opt_state = optimizer.update(grads, opt_state)
            params = optax.apply_updates(params, updates)
            batch_losses.append(loss_val)
    
        epoch_time = time.time() - t_epoch
        avg_loss = jnp.mean(jnp.array(batch_losses))
        test_acc = accuracy_fn(params, test_boards, test_labels)
        
        
        sys.stdout.write("\r" + " " * 50 + "\r") 
        sys.stdout.flush()
        print(f"Epoch {epoch}/{epochs} - T={epoch_time:.2f}s - Loss={avg_loss:.4f} - Test Acc={test_acc:.3f}")
    
    total_time = time.time() - start_all
    print(f"Time needed：{total_time:.1f}s")
    
    # 最终评估
    final_train_acc = accuracy_fn(params, train_boards, train_labels)
    final_test_acc = accuracy_fn(params, test_boards, test_labels)
    print(f"Final Train Acc: {final_train_acc:.4f}")
    print(f"Final Test  Acc: {final_test_acc:.4f}")

else:
    print("Not enough data")


成功加载 lightning.gpu 设备，将使用 NVIDIA cuQuantum 加速。
正在生成数据...
样本总数：6046
类别计数: {'draw': 4536, 'circle': 490, 'cross': 1020}
使用的 Class Weights: {'draw': 0.44429747207524983, 'circle': 4.112925170068027, 'cross': 1.9758169934640524}
训练集大小: 10887, 测试集大小: 1209
训练集类别分布: Counter({np.int32(0): 3629, np.int32(1): 3629, np.int32(2): 3629})
测试集类别分布: Counter({np.int32(1): 907, np.int32(2): 204, np.int32(0): 98})
JIT 编译中（第一次调用会比较慢）...
编译完成。开始训练。
Epoch 1/15 - T=1573.52s - Loss=2.2151 - Test Acc=0.145
Epoch 2/15 - Batch 2/85